In [ ]:
import pandas as pd

import os
from importlib import reload

from tqdm import tqdm
tqdm.pandas(desc="progress: ")

# Utility variable
import sys
sys.path.insert(0, '../..')

# var
import var.var as V

# utils
import utils.data as D
import utils.io as IO
import utils.preprocess as PP

## Read data

In [ ]:
df_applicants = D.read_df_applicants()
df_applicants.head()

In [ ]:
df_comments = D.read_df_comments()
split_comments = D.read_split_comments()

In [ ]:
df_comments.head()

In [ ]:
split_comments = D.read_split_comments()
split_comments_nsp = D.read_split_comments_nsp()

## Find train/test idx for `split_comments`

In [ ]:
train_sc_idx = []
test_sc_idx = []

ref = df_comments.split_comment.to_list()

i = 0
j = 0

for sc_idx, _sc in tqdm(enumerate(split_comments)):
    found_sc = False
    
    while not found_sc:
        ## empty comment
        if len(ref[i]) == 0:
            i += 1
            continue
            
        ## already iterate through one comment
        if j >= len(ref[i]):
            i += 1
            j = 0
            continue
        
        if _sc == ref[i][j]:
            _year = df_comments.iloc[i]['year']
            _id = df_comments.iloc[i]['id']
            
            train_or_test = df_applicants.query(
                '`year` == {} and `id` == {}'.format(_year, _id)
            )['train_or_test'].to_list()[0]
            
            if train_or_test == 'train':
                train_sc_idx.append(sc_idx)
                found_sc = True
            elif train_or_test == 'test':
                test_sc_idx.append(sc_idx)
                found_sc = True
                
        j += 1

In [ ]:
sc_train_test_list = [0] * len(split_comments)

for i in train_sc_idx:
    sc_train_test_list[i] = 'train'

for i in test_sc_idx:
    sc_train_test_list[i] = 'test'

## Find train/test idx for `split_comment_nsp`

In [ ]:
train_sc_nsp_idx = []
test_sc_nsp_idx = []

ref = df_comments.split_comment_nsp.to_list()

i = 0
j = 0

for sc_nsp_idx, _sc_nsp in tqdm(enumerate(split_comments_nsp)):
    found_sc_nsp = False

    while not found_sc_nsp:
        ## empty comment
        if len(ref[i]) == 0:
            i += 1
            continue
            
        ## already iterate through one comment
        if j >= len(ref[i]):
            i += 1
            j = 0
            continue
        
        if _sc_nsp == ref[i][j]:
            _year = df_comments.iloc[i]['year']
            _id = df_comments.iloc[i]['id']
            
            train_or_test = df_applicants.query(
                '`year` == {} and `id` == {}'.format(_year, _id)
            )['train_or_test'].to_list()[0]
            
            if train_or_test == 'train':
                train_sc_nsp_idx.append(sc_nsp_idx)
                found_sc_nsp = True
            elif train_or_test == 'test':
                test_sc_nsp_idx.append(sc_nsp_idx)
                found_sc_nsp = True
                
        j += 1

In [ ]:
sc_nsp_train_test_list = [0] * len(split_comments_nsp)

for i in train_sc_nsp_idx:
    sc_nsp_train_test_list[i] = 'train'

for i in test_sc_nsp_idx:
    sc_nsp_train_test_list[i] = 'test'

## Find train/test idx for `split_comments_no_duplicate`

In [ ]:
split_comments_no_duplicate = []

train_sc_nd_idx = []
test_sc_nd_idx = []
sc_nd_train_test_list = []

for sc, train_or_test in zip(split_comments, sc_train_test_list):
    if sc not in split_comments_no_duplicate:
        idx = len(split_comments_no_duplicate)
        
        if train_or_test == 'train':
            train_sc_nd_idx.append(idx)
            sc_nd_train_test_list.append('train')
        elif train_or_test == 'test':
            test_sc_nd_idx.append(idx)
            sc_nd_train_test_list.append('test')
        
        split_comments_no_duplicate.append(sc)

## Find train/test idx for `split_comments_nsp_no_duplicate`

In [ ]:
split_comments_nsp_no_duplicate = []

train_sc_nsp_nd_idx = []
test_sc_nsp_nd_idx = []
sc_nsp_nd_train_test_list = []

for sc_nsp, train_or_test in zip(split_comments_nsp, sc_nsp_train_test_list):
    if sc_nsp not in split_comments_nsp_no_duplicate:
        idx = len(split_comments_nsp_no_duplicate)
        
        if train_or_test == 'train':
            train_sc_nsp_nd_idx.append(idx)
            sc_nsp_nd_train_test_list.append('train')
        elif train_or_test == 'test':
            test_sc_nsp_nd_idx.append(idx)
            sc_nsp_nd_train_test_list.append('test')
        
        split_comments_nsp_no_duplicate.append(sc_nsp)

In [ ]:
split_comments_train_test_indices = {
    "split_comments": {
        "train_idx": train_sc_idx,
        "test_idx": test_sc_idx,
        "train_test_list": sc_train_test_list,
    },
    "split_comments_no_duplicate": {
        "train_idx": train_sc_nd_idx,
        "test_idx": test_sc_nd_idx,
        "train_test_list": sc_nd_train_test_list,
    },
    "split_comments_nsp": {
        "train_idx": train_sc_nsp_idx,
        "test_idx": test_sc_nsp_idx,
        "train_test_list": sc_nsp_train_test_list,
    },
    "split_comments_nsp_no_duplicate": {
        "train_idx": train_sc_nsp_nd_idx,
        "test_idx": test_sc_nsp_nd_idx,
        "train_test_list": sc_nsp_nd_train_test_list,
    },
}

## Write split comments

In [ ]:
D.write_split_comments_no_duplicate(split_comments_no_duplicate)
D.write_split_comments_nsp_no_duplicate(split_comments_nsp_no_duplicate)
D.write_split_comments_train_test_indices(split_comments_train_test_indices)